## LangChain tools
Tools in LangChain are external functions or services (e.g., search engine, calculator, API) that the agent can call while solving tasks to extend its capabilities beyond just text generation.

### Install libraries

In [1]:
!pip install -q langchain langchain-openai python-dotenv


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [2]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

### Simple tool (calculator)

In [3]:
from langchain.tools import tool

@tool
def multiply_numbers(x: int, y: int) -> int:
    """Multiplies two integers"""
    return x * y

print(multiply_numbers.invoke({"x": 6, "y": 7}))


42


### Combine tools with LLM

In [4]:
from langchain_classic.agents import initialize_agent, AgentType

tools = [multiply_numbers]

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# the model will decide itself that it needs to use multiply_numbers
agent.run("What is twice the number of people on Earth?")


/tmp/ipykernel_29992/3854547854.py:5: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the [LangGraph documentation](https://langchain-ai.github.io/langgraph/) as well as guides for [Migrating from AgentExecutor](https://python.langchain.com/docs/how_to/migrate_agent/) and LangGraph's [Pre-built ReAct agent](https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/).
  agent = initialize_agent(
/tmp/ipykernel_29992/3854547854.py:13: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain-classic 0.1.0 and will be removed in 1.0. Use `invoke` instead.
  agent.run("What is twice the number of people on Earth?")




> Entering new AgentExecutor chain...
Thought: To find twice the number of people on Earth, I need to know the current estimated population. As of 2023, the estimated global population is around 8 billion. Therefore, I will multiply 8 billion by 2.

Action:
```
{
  "action": "multiply_numbers",
  "action_input": {
    "x": 8000000000,
    "y": 2
  }
}
```

Observation: 16000000000
Thought:I have calculated twice the number of people on Earth, which is 16 billion. 

Action:
```
{
  "action": "Final Answer",
  "action_input": "Twice the number of people on Earth is 16 billion."
}
```


> Finished chain.


'Twice the number of people on Earth is 16 billion.'

### Tool - simple knowledge database

In [5]:
@tool
def company_info(name: str) -> str:
    """Returns basic company information based on name."""
    companies = {
        "OpenAI": "Artificial intelligence company, creator of ChatGPT.",
        "LangChain": "A library that makes it easy to build applications with language models.",
        "Tesla": "Manufacturer of electric cars and batteries."
    }
    return companies.get(name, "I don't know this company.")

tools = [company_info]
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

agent.run("Tell me what Tesla does.")




> Entering new AgentExecutor chain...
I need to gather information about Tesla to understand what the company does. 
Action: company_info
Action Input: "Tesla"
Observation: Manufacturer of electric cars and batteries.
Thought:I now know that Tesla is primarily involved in the manufacturing of electric cars and batteries. 
Final Answer: Tesla manufactures electric cars and batteries.

> Finished chain.


'Tesla manufactures electric cars and batteries.'

### Available tools in LangChain
List of available tools: https://python.langchain.com/docs/integrations/tools/

In [6]:
!pip install wikipedia tavily-python


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [7]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_classic.agents import initialize_agent, AgentType

# Initialize search tools
# Tavily API key: https://app.tavily.com/home
search = TavilySearchResults(max_results=3)

# Direct use of tools
query = "The latest news about the UK Prime Minister."
results = search.invoke(query)

print(f"Search results for query: '{query}'")
for i, result in enumerate(results, 1):
    print(f"\n{i}. {result.get('title', 'Missing title')}")
    print(f"   URL: {result.get('url', 'Missing URL')}")
    print(f"   Content: {result.get('content', 'Missing content')[:200]}...")

# List of tools with search engine
tools = [search]

# Agent with internet access
agent_with_search = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# Example query requiring internet search
response = agent_with_search.run(
    "Who is the current Prime Minister of UK and how old is he/she?"
)
print(f"\nAgent answer:\n{response}")

/tmp/ipykernel_29992/1989281160.py:6: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the `langchain-tavily package and should be used instead. To use it run `pip install -U `langchain-tavily` and import as `from `langchain_tavily import TavilySearch``.
  search = TavilySearchResults(max_results=3)


Search results for query: 'The latest news about the UK Prime Minister.'

1. British Government – Latest news and updates from 10 Downing ...
   URL: https://metro.co.uk/tag/british-government/
   Content: Prime Minister Sir Keir Starmer departs 10 Downing Street, London, to attend Prime Minister's Questions at the Houses of Parliament. Picture date: Wednesday October 22, 2025. PA Photo. Photo credit sh...

2. Prime Minister arrives at COP to deliver clean energy jobs as major ...
   URL: https://www.gov.uk/government/news/prime-minister-arrives-at-cop-to-deliver-clean-energy-jobs-as-major-deals-in-ports-and-industry-announced
   Content: “I won’t let down future generations. At COP30 I will continue to show UK leadership on the world stage to work with others to stand up for our values and our future.”

Today at COP30 the Prime Minist...

3. Britain's Prince William calls for optimism on environment at ...
   URL: https://www.reuters.com/sustainability/climate-energy/britains-prince-w

In [8]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

# Initialization tools Wikipedia
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

# Direct use
wiki_result = wikipedia.run("Python programming language")
print(f"Wikipedia - Python:\n{wiki_result[:300]}...")

# Agent with multiple tools
# Agent with access to multiple information sources
tools_extended = [search, wikipedia, company_info]

agent_multi_tool = initialize_agent(
    tools=tools_extended,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# Query requiring the use of various tools
response = agent_multi_tool.run(
    "Find information about Python on Wikipedia, then search for the latest articles about new features in Python 3.13"
)
print(f"\nAnswer:\n{response}")

Wikipedia - Python:
Page: Python (programming language)
Summary: Python is a high-level, general-purpose programming language. Its design philosophy emphasizes code readability with the use of significant indentation. Python is dynamically type-checked and garbage-collected. It supports multiple programming paradigms, ...


> Entering new AgentExecutor chain...
I will first gather general information about Python from Wikipedia to understand its background and features. After that, I will search for the latest articles regarding new features in Python 3.13. 

Action: wikipedia  
Action Input: Python (programming language)  
Observation: Page: Python (programming language)
Summary: Python is a high-level, general-purpose programming language. Its design philosophy emphasizes code readability with the use of significant indentation. Python is dynamically type-checked and garbage-collected. It supports multiple programming paradigms, including structured (particularly procedural), object-